# Find sub-array in another array

based on this Stackoverflow question:
- https://stackoverflow.com/q/73522465/1609514

## Solution for 2D arrays

In [1]:
# %load find_array_in_array_2d.py
import numpy as np


def find_array_in_array_2d(ar, ar2):

    # Find all matches with first element of ar2
    match_idx = np.nonzero(ar[:-ar2.shape[0]+1, :-ar2.shape[1]+1] == ar2[0, 0])

    # Check remaining indices of ar2
    for i, j in list(np.ndindex(ar2.shape))[1:]:

        # End if no possible matches left
        if len(match_idx[0]) == 0:
            break

        # Index into ar offset by i, j
        nz2 = (match_idx[0] + i, match_idx[1] + j)

        # Find remaining matches with selected element
        to_keep = np.nonzero(ar[nz2] == ar2[i, j])[0]
        match_idx = match_idx[0][to_keep], match_idx[1][to_keep]

    return match_idx

In [2]:
# Example from stackoverflow question
ar = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 1, 2],
 [1, 1, 0, 1, 1, 0, 1, 1, 1, 1]]
)

ar2 = np.zeros((2,2))

In [3]:
match_idx = find_array_in_array_2d(ar, ar2)
match_idx

(array([4]), array([4]))

In [4]:
assert(match_idx == (np.array([4]), np.array([4])))

In [5]:
# Test with a random 2d array
shape = (10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j = np.random.randint(9, size=2)
    ar[i:i+2, j:j+2] = 10
ar2 = 10 * np.ones((2, 2)).astype(int)

ar, ar2

(array([[ 3,  6,  5,  9,  9,  0,  3,  9, 10, 10],
        [ 2,  3,  9,  7, 10, 10,  7,  1, 10, 10],
        [ 9,  2, 10, 10, 10, 10, 10, 10,  2,  4],
        [ 9, 10, 10, 10, 10,  3, 10, 10, 10, 10],
        [ 6, 10, 10, 10, 10,  9, 10, 10, 10, 10],
        [10, 10,  6,  5,  3,  4, 10, 10, 10, 10],
        [10, 10,  3,  8,  6,  9, 10, 10,  3,  3],
        [10, 10, 10,  3,  3,  1,  7,  1,  4,  3],
        [10, 10, 10,  3,  8,  1,  4,  1,  3,  4],
        [ 4, 10, 10,  1,  6,  9,  9,  4,  3,  5]]),
 array([[10, 10],
        [10, 10]]))

In [6]:
match_idx = find_array_in_array_2d(ar, ar2)
match_idx

(array([0, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 6, 7, 7, 8]),
 array([8, 4, 2, 3, 6, 1, 2, 3, 6, 7, 8, 6, 7, 8, 0, 6, 0, 0, 1, 1]))

In [7]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2], ar2) for i, j in zip(*match_idx)]
assert(all(checks))

## N-dimensional version

In [8]:
from functools import reduce

def find_array_in_array(ar, ar2):

    # Find all matches with first element of ar2
    slices = tuple(slice(None, -s + 1) for s in ar2.shape)
    match_idx = np.nonzero(ar[slices] == ar2.flat[0])
    
    # ar[:-ar2.shape[0]:, :-ar2.shape[1]]

    # Check remaining indices of ar2
    for idx in list(np.ndindex(ar2.shape))[1:]:

        if len(match_idx[0]) == 0:
            break

        # Index into ar offset by idx
        nz2 = tuple(m + i for m, i in zip(match_idx, idx))

        # Find remaining matches with selected element
        to_keep = np.nonzero(ar[nz2] == ar2[idx])[0]
        match_idx = tuple(m[to_keep] for m in match_idx)

    return match_idx

# 1d
assert(find_array_in_array(np.arange(0, 10), np.array([3, 4])) == (3,))
assert(find_array_in_array(np.arange(0, 10), np.array([4, -1]))[0].shape == (0,))

In [9]:
# Test with a random 2d array
shape = (10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j = np.random.randint(9, size=2)
    ar[i:i+2, j:j+2] = 10
ar2 = 10 * np.ones((2, 2)).astype(int)

ar, ar2

(array([[ 4,  1, 10, 10, 10, 10,  5,  8,  9,  9],
        [ 9,  6, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 4,  6,  7,  9,  8, 10, 10, 10, 10, 10],
        [ 6,  8,  7,  1, 10, 10, 10, 10,  3,  5],
        [10, 10,  2, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10,  2, 10, 10],
        [ 9,  4, 10, 10,  9, 10, 10, 10, 10, 10],
        [ 0,  1,  8, 10, 10,  4,  5, 10, 10, 10],
        [ 5,  0,  9, 10, 10,  3,  2,  8,  7,  6],
        [ 2,  5,  9,  6,  6,  0,  1,  0,  9,  2]]),
 array([[10, 10],
        [10, 10]]))

In [10]:
match_idx = find_array_in_array(ar, ar2)
match_idx

(array([0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 6, 6,
        7]),
 array([2, 3, 4, 5, 6, 7, 8, 5, 6, 4, 5, 6, 0, 3, 4, 5, 8, 2, 5, 8, 7, 8,
        3]))

In [11]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2], ar2) for i, j in zip(*match_idx)]
assert(all(checks))

In [12]:
# Test with a random 3d array
shape = (10, 10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j, k = np.random.randint(10, size=3)
    ar[i:i+2, j:j+2, k:k+2] = 10
ar2 = 10 * np.ones((2, 2, 2)).astype(int)

In [13]:
match_idx = find_array_in_array(ar, ar2)
match_idx

(array([1, 2, 2, 3, 4, 4, 5, 7, 7, 8, 8]),
 array([6, 0, 4, 4, 5, 7, 8, 1, 5, 0, 7]),
 array([8, 5, 4, 1, 5, 6, 8, 6, 6, 5, 7]))

In [14]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2, k:k+2], ar2) for i, j, k in zip(*match_idx)]
assert(all(checks))

## Solution using tools from Scikit-Image

In [15]:
from skimage.util import view_as_windows

def find_array_in_array_2d_ski(ar, ar2):
    view_ = view_as_windows(ar, ar2.shape)
    res_temp = np.all((view_ == ar2[None, ...]), (-2, -1))
    return np.nonzero(res_temp)

# 1d
assert(find_array_in_array(np.arange(0, 10), np.array([3, 4])) == (3,))
assert(find_array_in_array(np.arange(0, 10), np.array([4, -1]))[0].shape == (0,))

In [16]:
# Test with a random 2d array
shape = (10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j = np.random.randint(9, size=2)
    ar[i:i+2, j:j+2] = 10
ar2 = 10 * np.ones((2, 2)).astype(int)

In [17]:
match_idx = find_array_in_array(ar, ar2)
match_idx

(array([0, 0, 1, 2, 2, 2, 2, 2, 3, 3, 5, 6, 6, 6, 6, 7, 8]),
 array([2, 3, 2, 2, 5, 6, 7, 8, 3, 8, 2, 0, 1, 2, 6, 2, 4]))

In [18]:
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2], ar2) for i, j in zip(*match_idx)]
assert(all(checks))

In [19]:
# Test with a random 3d array
shape = (10, 10, 10)
ar = np.random.randint(10, size=np.product(shape)).reshape(shape)
for _ in range(15):
    i, j, k = np.random.randint(10, size=3)
    ar[i:i+2, j:j+2, k:k+2] = 10
ar2 = 10 * np.ones((2, 2, 2)).astype(int)

match_idx = find_array_in_array(ar, ar2)
# Check these are correct
checks = [np.array_equal(ar[i:i+2, j:j+2, k:k+2], ar2) for i, j, k in zip(*match_idx)]
assert(all(checks))

## Speed tests

In [20]:
# Test with a large random 2d array
shape = (1000, 1000)
ar = np.random.randint(50, size=np.product(shape)).reshape(shape)
for _ in range(50):
    i, j = np.random.randint(9, size=2)
    ar[i:i+5, j:j+5] = 50
ar2 = 50 * np.ones((5, 5)).astype(int)

In [21]:
assert(np.array_equal(
    find_array_in_array_2d(ar, ar2),
    find_array_in_array(ar, ar2)
))

In [22]:
assert(np.array_equal(
    find_array_in_array_2d(ar, ar2),
    find_array_in_array_2d_ski(ar, ar2)
))

In [23]:
%timeit find_array_in_array_2d(ar, ar2)

5.56 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit find_array_in_array(ar, ar2)

5.1 ms ± 91.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit find_array_in_array_2d_ski(ar, ar2)

200 ms ± 3.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
